# https://app.asana.com/0/0/1199347781908598/f

In [ ]:
from taigapy import TaigaClient
tc = TaigaClient()

In [ ]:
CCLE_expression = tc.get(name='dmc-20q4-fcf4', version=33, file='CCLE_expression')

In [ ]:
CCLE_expression.loc[:, CCLE_expression.columns.str.startswith('PINX1')]

In [ ]:
CCLE_expression_full = tc.get(name='dmc-20q4-fcf4', version=33, file='CCLE_expression_full')

In [ ]:
CCLE_expression_full.loc[:, CCLE_expression_full.columns.str.startswith('PINX1')]

In [ ]:
import numpy as np
np.log2(CCLE_expression_full.loc[:, CCLE_expression_full.columns.str.startswith('PINX1')]+1)

# https://forum.depmap.org/t/high-number-of-cell-lines-with-copy-number-equal-to-0/295/5

In [ ]:
from taigapy import TaigaClient
tc = TaigaClient()

In [ ]:
CCLE_gene_cn_20q4 = tc.get(name='public-20q4-a4b3', version=39, file='CCLE_gene_cn')

In [ ]:
CCLE_segment_cn_20q4 = tc.get(name='public-20q4-a4b3', version=39, file='CCLE_segment_cn')

In [ ]:
CCLE_gene_cn_20q3 = tc.get(name='public-20q3-3d35', version=33, file='CCLE_gene_cn')

In [ ]:
CCLE_segment_cn_20q3 = tc.get(name='public-20q3-3d35', version=33, file='CCLE_segment_cn')

In [ ]:
CCLE_gene_cn_20q4.columns[CCLE_gene_cn_20q4.columns.str.startswith('ERBB2')]

In [ ]:
CCLE_gene_cn_20q4.loc['ACH-000859', 'ERBB2 (2064)'], CCLE_gene_cn_20q3.loc['ACH-000859', 'ERBB2 (2064)']

In [ ]:
#ERBB2: chr17:39,687,914-39,730,426(GRCh38/hg38)

In [ ]:
tmp = CCLE_segment_cn_20q4[CCLE_segment_cn_20q4['DepMap_ID'] == 'ACH-000859']
tmp = tmp[tmp['Chromosome'] == '17']
tmp[(tmp['Start'] < 39730426) & (tmp['End']>39687914)]

### why is there so much difference in gene level copy numbers

In [ ]:
tmp1 = CCLE_gene_cn_20q3.stack()
tmp2 = CCLE_gene_cn_20q4.stack()

In [ ]:
import pandas as pd
CCLE_gene_cn_20q3q4 = pd.concat([tmp1, tmp2], axis=1)

In [ ]:
CCLE_gene_cn_20q3q4.columns=['20q3', '20q4']

In [ ]:
tmp_peak_at1 = CCLE_gene_cn_20q3q4[(CCLE_gene_cn_20q3q4['20q4']>2) & (CCLE_gene_cn_20q3q4['20q3'] > 0.5) & (CCLE_gene_cn_20q3q4['20q3']<1.5)]

In [ ]:
len(set(CCLE_gene_cn_20q3q4.index.get_level_values(0)) - set(tmp_peak_at1.index.get_level_values(0)))

In [ ]:
len(set(tmp_peak_at1.index.get_level_values(0)))

In [ ]:
CCLE_gene_cn_20q3q4.reset_index(inplace=True)
CCLE_gene_cn_20q3q4.rename(columns={'level_0': 'DepMap_ID', 'level_1': 'gene'}, inplace=True)

In [ ]:
# cols = ['Source_20q3_x', 'Source_20q4_x',
#        'source_change_x', 'Source_20q3_y', 'Source_20q4_y', 'source_change_y']

# CCLE_gene_cn_20q3q4.drop(cols, axis=1, inplace=True)

In [ ]:
sources = pd.merge(CCLE_segment_cn_20q3[['DepMap_ID', 'Source']].drop_duplicates(), 
         CCLE_segment_cn_20q4[['DepMap_ID', 'Source']].drop_duplicates(), 
         on='DepMap_ID', suffixes=['_20q3', '_20q4'])

sources['source_change'] = sources.apply(lambda x: '{:s} -> {:s}'.format(x['Source_20q3'], x['Source_20q4']), axis=1)
sources['source_has_changed'] = (sources['Source_20q3'] != sources['Source_20q4'])

In [ ]:
CCLE_gene_cn_20q3q4 = pd.merge(CCLE_gene_cn_20q3q4, sources, on='DepMap_ID')

In [ ]:
CCLE_gene_cn_20q3q4

In [ ]:
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt
plt.figure(figsize=(20,10))
sns.scatterplot(data=CCLE_gene_cn_20q3q4.sample(1000000, random_state=0), x='20q3', y='20q4', 
                hue='source_change', style='source_has_changed', alpha=0.5, cmap='Tab20')

What's going on with the peak?

In [ ]:
tmp = CCLE_gene_cn_20q3q4[(CCLE_gene_cn_20q3q4['20q3'] > 0.8) & (CCLE_gene_cn_20q3q4['20q3'] < 1.2)]

In [ ]:
tmp[tmp['20q4'] > 5]

Looks like a lot of the genes in the 20q3 ~ 1 peak are on X chromosome. Let's remove XY genes and look at results again.

In [ ]:
genemapping_19Q1 = pd.read_csv('/Users/jnoorbak/broad/projects/depmap-omics/ccle_processing/data/genemapping_19Q1.csv')
genemapping_19Q1

In [ ]:
genemapping_19Q1_noXY = genemapping_19Q1[~genemapping_19Q1['Chromosome'].isin(['X', 'Y'])]

In [ ]:
genenames_noXY = set(genemapping_19Q1_noXY['symbol'])

In [ ]:
genemapping_19Q1_noXY

In [ ]:
CCLE_gene_cn_20q3q4_filter = CCLE_gene_cn_20q3q4['gene'].map(lambda x: x.split()[0] in genenames_noXY)

In [ ]:
CCLE_gene_cn_20q3q4_noXY = CCLE_gene_cn_20q3q4[CCLE_gene_cn_20q3q4_filter]

In [ ]:
tmp = CCLE_gene_cn_20q3q4_noXY[(CCLE_gene_cn_20q3q4_noXY['20q3'] > 0.8) & (CCLE_gene_cn_20q3q4_noXY['20q3'] < 1.2)]

In [ ]:
tmp[tmp['20q4'] > 3]

In [ ]:
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt
plt.figure(figsize=(20,10))
sns.scatterplot(data=CCLE_gene_cn_20q3q4_noXY.sample(1000000, random_state=0), x='20q3', y='20q4', 
                hue='source_change', style='source_has_changed', alpha=0.5, cmap='Tab20')

There are a lot of SNP array data that were changed to other data. Let's drop those and check again:

In [ ]:
CCLE_gene_cn_20q3q4_noXY_noSNP = CCLE_gene_cn_20q3q4_noXY[CCLE_gene_cn_20q3q4_noXY['Source_20q3'] != 'Broad SNP' ]

In [ ]:
plt.figure(figsize=(20,10))
sns.scatterplot(data=CCLE_gene_cn_20q3q4_noXY_noSNP.sample(1000000, random_state=0), x='20q3', y='20q4', 
                hue='source_change', style='source_has_changed', alpha=0.5, cmap='Tab20')

Let's drop cases with no source change for cleaner plots

In [ ]:
CCLE_gene_cn_20q3q4_noXY_sourcechanged = CCLE_gene_cn_20q3q4_noXY[CCLE_gene_cn_20q3q4_noXY['source_has_changed']]

In [ ]:
plt.figure(figsize=(20,10))
sns.scatterplot(data=CCLE_gene_cn_20q3q4_noXY_sourcechanged.sample(1000000, random_state=0), x='20q3', y='20q4', 
                hue='source_change', alpha=0.5, cmap='Tab20')

Let's only keep cases with no source change

In [ ]:
CCLE_gene_cn_20q3q4_noXY_nosourcechanged = CCLE_gene_cn_20q3q4_noXY[~CCLE_gene_cn_20q3q4_noXY['source_has_changed']]

In [ ]:
plt.figure(figsize=(20,10))
sns.scatterplot(data=CCLE_gene_cn_20q3q4_noXY_nosourcechanged.sample(1000000, random_state=0), x='20q3', y='20q4', 
                hue='source_change', alpha=0.1, cmap='Tab20')

Let's only keep cases with no source change (but add back XY)

In [ ]:
CCLE_gene_cn_20q3q4_nosourcechanged = CCLE_gene_cn_20q3q4[~CCLE_gene_cn_20q3q4['source_has_changed']]

In [ ]:
plt.figure(figsize=(20,10))
sns.scatterplot(data=CCLE_gene_cn_20q3q4_nosourcechanged.sample(1000000, random_state=0), x='20q3', y='20q4', 
                hue='source_change', alpha=0.1, cmap='Tab20')

Which samples have the unusual log transform?

In [ ]:
import numpy as np
weird_curve = np.abs(CCLE_gene_cn_20q3q4['20q4'] - np.log2(CCLE_gene_cn_20q3q4['20q3']+1))

In [ ]:
CCLE_gene_cn_20q3q4_weird_curve = CCLE_gene_cn_20q3q4[weird_curve < 1e-9]

In [ ]:
plt.figure(figsize=(20,10))
sns.scatterplot(data=CCLE_gene_cn_20q3q4_weird_curve.sample(1000000, random_state=0), x='20q3', y='20q4', 
                hue='source_change', style='source_has_changed', alpha=0.5, cmap='Tab20')

In [ ]:
CCLE_gene_cn_20q3q4['weird_curve'] = weird_curve < 1e-9

In [ ]:
mean_genes_in_weird_curve = CCLE_gene_cn_20q3q4.groupby('DepMap_ID')['weird_curve'].mean()

Let's plot the ratio of genes in each sample that are

In [ ]:
mean_genes_in_weird_curve.plot.hist(40)

In [ ]:
(mean_genes_in_weird_curve > 0.8).sum()

Looks like samples are either in the curve or not. 862 samples are on this curve.

How about segment files?

In [ ]:
CCLE_segment_cn_20q4 = tc.get(name='public-20q4-a4b3', version=39, file='CCLE_segment_cn')

In [ ]:
CCLE_segment_cn_20q4.sort_values(['DepMap_ID', 'Chromosome', 'Start', 'End'], inplace=True)

# https://forum.depmap.org/t/disappearing-mutations/248

In [ ]:
from taigapy import TaigaClient
tc = TaigaClient()


In [ ]:
depmap_mutation_calls_20Q4 = tc.get(name='mutations-latest-ed72', version=2, file='wes_somatic_mutations_all_20Q4')
depmap_mutation_calls_20Q2 = tc.get(name='depmap-mutation-calls-9be3', version=22, file='depmap_20Q2_mutation_calls')
depmap_mutation_calls_20Q1 = tc.get(name='public-20q1-c3b6', version=17, file='CCLE_mutations')
depmap_mutation_calls_19Q2 = tc.get(name='internal-19q2-9504', version=24, file='CCLE_mutations')

In [ ]:
tmp = depmap_mutation_calls_20Q2.copy()
tmp = tmp[tmp['DepMap_ID']=='ACH-000158']
tmp[tmp['Hugo_Symbol']=='MYD88']

In [ ]:
tmp = depmap_mutation_calls_20Q1.copy()
tmp = tmp[tmp['DepMap_ID']=='ACH-000158']
tmp[tmp['Hugo_Symbol']=='MYD88']

In [ ]:
tmp = depmap_mutation_calls_19Q2.copy()
tmp = tmp[tmp['DepMap_ID']=='ACH-000158']
tmp[tmp['Hugo_Symbol']=='MYD88']

In [ ]:
wes_mutation_somatic_unfiltered_withreplicates = tc.get(name='mutations-latest-ed72', version=2, 
                                                        file='wes_mutation_somatic_unfiltered_withreplicates')

In [ ]:
tmp = wes_mutation_somatic_unfiltered_withreplicates
tmp = tmp[tmp['DepMap_ID']=='ACH-000158']
tmp[tmp['Hugo_Symbol']=='MYD88']

# Scratch

In [ ]:
gene_sets_20Q4_all = tc.get(name='expression-d035', version=2, file='gene_sets_20Q4_all')


In [ ]:
gene_sets_20Q4_all

In [ ]:
from taigapy import TaigaClient
tc = TaigaClient()

depmap_mutation_calls = tc.get(name='mutations-latest-ed72', version=2, file='wes_somatic_mutations_all_20Q4')
depmap_mutation_calls_old = tc.get(name='depmap-mutation-calls-9be3', version=22, file='depmap_20Q2_mutation_calls')


In [ ]:
from gsheets import Sheets
sheets_obj = Sheets.from_files('~/.client_secret.json', '~/.storage.json')

sample_tracker = 'https://docs.google.com/spreadsheets/d/1XkZypRuOEXzNLxVk9EOHeWRE98Z8_DBvL4PovyM01FE/edit#gid=892927125'
sheets = sheets_obj.get(sample_tracker).sheets
# tmp1 = sheets[3].to_frame(header=0, index_col=None, usecols=['Sample ID', 'Primary Disease', 'Final Diagnosis', 'Fingerprint Pass / Fail', 'TCGA code (Moony)'])

depmap_samples = sheets[0].to_frame(header=0, index_col=0)


In [ ]:
depmap_samples['baits']

In [ ]:
tmp = depmap_mutation_calls[depmap_mutation_calls['Hugo_Symbol']=='MYD88']['DepMap_ID'].sort_values()
tmp

In [ ]:
ccle_lines = depmap_samples[depmap_samples['source'] == 'CCLE2']['arxspan_id'].drop_duplicates().values

In [ ]:
len(ccle_lines)

In [ ]:
import pandas as pd
fp = pd.read_csv('../other/Fingerprinting_test - parental_ref.tsv', sep='\t', header=1)

In [ ]:
len(fp)

In [ ]:
fp = fp[fp['arxspan_id'].isin(ccle_lines)]

In [ ]:
len(fp)

In [ ]:
fp.to_csv('../other/fp_ccle.tsv', sep='\t', index=None)

In [ ]:
!open ../other/

In [ ]:
depmap_mutation_calls[depmap_mutation_calls['Hugo_Symbol']=='MYD88']

In [ ]:
depmap_samples['baits'].loc[tmp].value_counts()

In [ ]:
depmap_mutation_calls_old[depmap_mutation_calls_old['Hugo_Symbol']=='MYD88']['DepMap_ID'].sort_values()

In [ ]:
CCLE_fusions = tc.get(name='internal-20q2-7f46', version=18, file='CCLE_fusions')

In [ ]:
CCLE_fusions

In [ ]:
from taigapy import TaigaClient
tc = TaigaClient()
dmc_20Q3_gene_cn = tc.get(name='depmap-cn-data-9b9d', version=15, file='dmc_20Q3_gene_cn')
dmc_20Q3_segs_cn = tc.get(name='depmap-cn-data-9b9d', version=15, file='dmc_20Q3_segs_cn')

In [ ]:
dmc_20Q3_segs_cn

In [ ]:
internal_20Q3_gene_cn = tc.get(name='depmap-wes-cn-data-81a7', version=24, file='internal_20Q3_gene_cn')

In [ ]:
internal_20Q3_segs_cn = tc.get(name='depmap-wes-cn-data-81a7', version=24, file='internal_20Q3_segs_cn')

In [ ]:
%%bash
# maf=gs://fc-secure-d8458bee-4f7e-4de1-b9d0-17df5cbe9477/e2d8d510-ff89-4825-9d23-51b026a2f6e0/CNVSomaticPairWorkflow/e76c5c17-5528-4ec1-b44a-b74b12d5bcc1/call-CallCopyRatioSegmentsTumor/AA01-Tumor-SM-5GU8S.hg38.called.seg
maf=gs://fc-secure-d8458bee-4f7e-4de1-b9d0-17df5cbe9477/e2d8d510-ff89-4825-9d23-51b026a2f6e0/CNVSomaticPairWorkflow/fc082402-b099-4035-8bd5-f13cc87207af/call-CallCopyRatioSegmentsTumor/AA02-Tumor-SM-5GU8T.hg38.called.seg
gsutil cp $maf /tmp/    

In [ ]:
cnv=pd.read_csv('/tmp/AA02-Tumor-SM-5GU8T.hg38.called.seg', sep='\t', comment='@')

In [ ]:
cnv=pd.read_csv('/tmp/AA01-Tumor-SM-5GU8S.hg38.called.seg', sep='\t', comment='@')

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style('whitegrid')
cnv['CRatio']=2**cnv['MEAN_LOG2_COPY_RATIO']
sns.stripplot(data=cnv, x='CALL', y='CRatio', order=['-', '0', '+'])


In [ ]:
from taigapy import TaigaClient
tc = TaigaClient()

CCLE_segment_cn = tc.get(name='internal-20q2-7f46', version=18, file='CCLE_segment_cn')

In [ ]:
CCLE_segment_cn

In [ ]:
import numpy as np
cnv.groupby('CALL')['MEAN_LOG2_COPY_RATIO'].apply(lambda x: (2**x).describe()).unstack().loc[['-', '0', '+']]

In [ ]:
import pandas as pd

df = pd.read_csv('/tmp/tmp.txt', sep='\t')

In [ ]:
df['COSMIC_overlapping_mutations'].notnull().count()

In [ ]:
df['COSMIC_n_overlapping_mutations'].notnull().count()

In [ ]:
df[df['COSMIC_n_overlapping_mutations'].notnull()][['Hugo_Symbol', 'COSMIC_overlapping_mutations', 'COSMIC_n_overlapping_mutations']]

In [ ]:
df[df['COSMIC_n_overlapping_mutations'] == 1582][['Hugo_Symbol', 'COSMIC_overlapping_mutations']].iloc[0]

In [ ]:
df['COSMIC_overlapping_mutations'].unique()

In [ ]:
%load_ext rpy2.ipython

In [ ]:
%%R
COSMIChotspotMinCnt = 10
maf <- read.csv(file = '/tmp/tmp.txt', sep='\t')
# Cosmic cnts (overlapping muts)

COSMICcnt = unlist(sapply(as.character(maf$COSMIC_overlapping_mutations), 
                          function(x){sum(as.numeric(
                            gsub(')','', sapply(unlist(strsplit(x, '|', fixed=TRUE)), 
                                                function(x){ unlist(strsplit(x,'(', fixed=TRUE))[2] }))))}));



if(is.null(maf$COSMIC_overlapping_mutations)){
  COSMICcnt=  maf$COSMIC_n_overlapping_mutations
  if(is.null(COSMICcnt)){
    COSMICcnt =  maf$i_COSMIC_n_overlapping_mutations
    
  }
}


# COSMICcnt= convertToNumeric(COSMICcnt)
# isCOSMIChotspot = (COSMICcnt>=COSMIChotspotMinCnt);
# COSMICcnt

In [ ]:
from taigapy import TaigaClient
tc = TaigaClient()
internal_20Q2_gene_cn = tc.get(name='depmap-wes-cn-data-81a7', version=22, file='internal_20Q2_gene_cn')
internal_20Q2_segs_cn = tc.get(name='depmap-wes-cn-data-81a7', version=22, file='internal_20Q2_segs_cn')

In [ ]:
tmp = (2**internal_20Q2_gene_cn).unstack()

In [ ]:
tmp

In [ ]:
(2*(tmp-1)).describe()

In [ ]:
internal_20Q2_gene_cn.mean()

In [ ]:
internal_20Q2_gene_cn.median()

In [ ]:
internal_20Q2_segs_cn.groupby('DepMap_ID')['Segment_Mean'].median().describe()

In [ ]:
(np.log2(internal_20Q2_segs_cn.groupby('DepMap_ID')['Segment_Mean'].min())).plot.hist(bins=40)

In [ ]:
internal_20Q2_segs_cn.groupby('DepMap_ID')['Segment_Mean'].mean().describe()

In [ ]:
2**.005

In [ ]:
internal_20Q2_segs_cn['Segment_Mean'].describe()

In [ ]:
internal_20Q2_gene_cn.stack().describe()

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
internal_20Q2_segs_cn['Segment_Mean'].sample(1000).plot.hist(bins=100)
plt.xlim([0, 5]);

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
tmp.sample(1000).plot.hist(bins=100)
plt.xlim([0, 5]);

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
internal_20Q2_gene_cn.unstack().sample(1000).plot.hist(bins=100)
plt.xlim([0, 5]);

In [ ]:
import numpy as np
x = np.linspace(-10, 20, 100)
y = np.log2(2**x+1)

plt.plot(x, y)
plt.plot(x, x, '--')